In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import(precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, mean_absolute_error)
from sklearn.preprocessing import Normalizer
from sklearn import model_selection as ms
from sklearn import preprocessing as pp
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers import Dense, Dropout,Flatten, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import h5py

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Deep CNN Model/results/cnn_deep_mawi1.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [5]:
# get information from MAWILab Dataset

class dataset:
    mawi_train_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels(001).pkl")
    mawi_test_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels(002).pkl")

class preprocess:
    output_columns_2labels = ['status_anomaly','status_normal']

    x_input = dataset.mawi_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.mawi_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.mawi_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.mawi_test_2labels.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (40934, 4)
y_train shape: (40934, 2)
x_test shape: (47265, 4)
y_test shape: (47265, 2)


In [6]:
#reshape data
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (40934, 4, 1)
test shape after reshape: (47265, 4, 1)


In [7]:
#size of parameters
batch_size = 64
num_classes = 2
epochs = 20
filter_size=3
droprate=0.50

In [8]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(4, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))

#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), padding="same",
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#convolution 3rd layer
model.add(Conv1D(256, kernel_size=(filter_size), padding="same",
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#Flatten
model.add(Flatten())

#FCN Layer
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN Layer
model.add(Dense(32,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN Layer
model.add(Dense(16,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4, 64)             256       
                                                                 
 batch_normalization (BatchN  (None, 4, 64)            256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 4, 64)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 128)            24704     
                                                                 
 batch_normalization_1 (Batc  (None, 4, 128)           512       
 hNormalization)                                                 
                                                  

In [9]:
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 40934 samples, validate on 47265 samples
Epoch 1/20
40934/40934 [==============================] - ETA: 0s - loss: 0.4893 - acc: 0.7977

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


40934/40934 [==============================] - 25s 608us/sample - loss: 0.4893 - acc: 0.7977 - val_loss: 0.4619 - val_acc: 0.7257
Epoch 2/20
40934/40934 [==============================] - 12s 282us/sample - loss: 0.3957 - acc: 0.8252 - val_loss: 0.4649 - val_acc: 0.7257
Epoch 3/20
40934/40934 [==============================] - 11s 279us/sample - loss: 0.3828 - acc: 0.8252 - val_loss: 0.4670 - val_acc: 0.7257
Epoch 4/20
40934/40934 [==============================] - 11s 277us/sample - loss: 0.3772 - acc: 0.8252 - val_loss: 0.4721 - val_acc: 0.7257
Epoch 5/20
40934/40934 [==============================] - 11s 281us/sample - loss: 0.3745 - acc: 0.8255 - val_loss: 0.4682 - val_acc: 0.7257
Epoch 6/20
40934/40934 [==============================] - 12s 282us/sample - loss: 0.3724 - acc: 0.8265 - val_loss: 0.4651 - val_acc: 0.7257
Epoch 7/20
40934/40934 [==============================] - 12s 284us/sample - loss: 0.3710 - acc: 0.8262 - val_loss: 0.4666 - val_acc: 0.7257
Epoch 8/20
40934/40934 [

In [10]:
#Confusion Matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)

target_name=["is_Attack", "is_Normal"]

print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_name, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

   is_Attack     0.7925    0.0457    0.0864     12963
   is_Normal     0.7341    0.9955    0.8450     34302

    accuracy                         0.7350     47265
   macro avg     0.7633    0.5206    0.4657     47265
weighted avg     0.7501    0.7350    0.6369     47265

[[  592 12371]
 [  155 34147]]
